In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from io import StringIO
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from dataset import *
from torch.utils.data import DataLoader

In [3]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cuda:2


In [4]:
if torch.cuda.is_available():
    print (torch.cuda.current_device())
    torch.cuda.set_device(device)
    print (torch.cuda.current_device())

0
2


In [5]:
torch.set_printoptions(sci_mode=False)

In [6]:
root_folder = 'data/simple_reacher/training'
number_of_trajectories = 5000
batch_size = 32

In [39]:
root_folder = 'data/simple_reacher/sample_data_all_limited_joints'
number_of_trajectories = 5

In [6]:
start = time.time()
base = SimpleReacherBaseDataset(root_folder, number_of_trajectories, device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 4.9346020221710205 sec


In [41]:
start = time.time()
ondemand = SimpleReacherOnDemandDataset(root_folder, number_of_trajectories)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.07968378067016602 sec


In [5]:
start = time.time()
preloaded = SimpleReacherPreLoadedDataset(root_folder, number_of_trajectories)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 10.221627950668335 sec


In [6]:
ondemand.trajectory_data.shape

(2194, 9)

In [7]:
len(ondemand)

2194

In [8]:
print(preloaded.trajectory_data.shape)
print(preloaded.image_data.shape)

(2194, 9)
(2194, 62500)


In [9]:
idx = np.random.randint(len(ondemand.trajectory_data), size=(batch_size))

In [10]:
idx

array([ 519, 1346,  545,  451,  926, 1596, 1284, 1450,  408,  921, 1548,
       1210,  641,  618, 1954, 1747, 2140, 1758,  242, 1181,  180, 1143,
        334, 1216, 1971, 2152, 2148,  542,   86, 1576,   22,  409])

In [11]:
start = time.time()
ondemandBatch = ondemand[idx]
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.10364007949829102 sec


In [12]:
start = time.time()
preloadedBatch = preloaded[idx]
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 0.001249074935913086 sec


In [13]:
np.all(ondemandBatch['images'] == preloadedBatch['images'])

True

In [14]:
onedemandLoader = DataLoader(ondemand, batch_size=batch_size, shuffle=True)
preloadedLoader = DataLoader(preloaded, batch_size=batch_size, shuffle=True)

In [15]:
epochs = 1

In [16]:
def simulate_training(dataLoader):
    start = time.time()
    builder = StringIO()
    for e in range(epochs):
        for i, data in enumerate(dataLoader):
            builder.write('Epoch {}: Running batch index = {} with data shape = {}'.format(e, i, data['images'].shape))
            builder.write('\n')
            break
    print ("Time Taken: {} sec".format(time.time() - start))
    log = builder.getvalue()
    builder.close()
    return log

In [17]:
log1 = simulate_training(onedemandLoader)

Time Taken: 0.2524890899658203 sec


In [18]:
log1

'Epoch 0: Running batch index = 0 with data shape = torch.Size([32, 62500])\n'

In [19]:
log2 = simulate_training(preloadedLoader)

Time Taken: 0.005193948745727539 sec


In [20]:
log2

'Epoch 0: Running batch index = 0 with data shape = torch.Size([32, 62500])\n'

In [29]:
sys.getsizeof(ondemand.trajectory_data)

158080

In [27]:
sys.getsizeof(preloaded.image_data)

137125112

In [7]:
start = time.time()
train_data = SimpleReacherOnDemandDataset(root_folder, range(4750), device=None)
test_data = SimpleReacherOnDemandDataset(root_folder, range(4750, 5000), device=None)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 21.97927212715149 sec


In [7]:
train_data = SimpleReacherOnDemandDataset(root_folder, range(5000), device=device)

In [8]:
print (train_data.trajectory_data.shape)

torch.Size([108673, 9])


In [9]:
print (torch.min(train_data.trajectory_data[:, :1]))
print (torch.max(train_data.trajectory_data[:, :1]))
print (torch.min(train_data.trajectory_data[:, 1:3]))
print (torch.max(train_data.trajectory_data[:, 1:3]))
print (torch.min(train_data.trajectory_data[:, 3:6]))
print (torch.max(train_data.trajectory_data[:, 3:6]))
print (torch.min(train_data.trajectory_data[:, 6:]))
print (torch.max(train_data.trajectory_data[:, 6:]))
# print (torch.min(test_data.trajectory_data[:, :3]))
# print (torch.max(test_data.trajectory_data[:, :3]))
# print (torch.min(test_data.trajectory_data[:, 3:6]))
# print (torch.max(test_data.trajectory_data[:, 3:6]))
# print (torch.min(test_data.trajectory_data[:, 6:]))
# print (torch.max(test_data.trajectory_data[:, 6:]))
print (np.pi)

tensor(-3.1352, device='cuda:2', dtype=torch.float64)
tensor(3.1405, device='cuda:2', dtype=torch.float64)
tensor(-2.8946, device='cuda:2', dtype=torch.float64)
tensor(2.8969, device='cuda:2', dtype=torch.float64)
tensor(-3.1364, device='cuda:2', dtype=torch.float64)
tensor(3.1412, device='cuda:2', dtype=torch.float64)
tensor(-0.1990, device='cuda:2', dtype=torch.float64)
tensor(0.1985, device='cuda:2', dtype=torch.float64)
3.141592653589793


In [10]:
actions = train_data.trajectory_data[:, 6:]
print (actions.shape)
print (torch.max(actions))
print (torch.min(actions))
print (torch.where(torch.abs(actions) <= 0.15)[0].shape)
print (torch.where(torch.abs(actions) > 0.15)[0].shape)
print (torch.unique(torch.where(actions > 0.15)[0]))

torch.Size([108673, 3])
tensor(0.1985, device='cuda:2', dtype=torch.float64)
tensor(-0.1990, device='cuda:2', dtype=torch.float64)
torch.Size([325294])
torch.Size([725])
tensor([  3035,   3036,   3037,   3038,   3039,   3040,   3041,   3042,   4013,
          4014,   4015,   7230,   7768,   7769,   7770,   8058,   8059,   8060,
          8061,   8062,   8063,   8064,   8065,   8066,   8067,   8820,   8821,
          8822,   8823,   8824,  10450,  10451,  10452,  10453,  10454,  10455,
         10506,  10507,  10508,  10509,  10510,  10511,  10656,  10657,  10658,
         10659,  10660,  10661,  10662,  10696,  10697,  10698,  10699,  10700,
         12483,  12484,  12485,  12486,  13834,  13835,  14230,  14231,  14232,
         15638,  15639,  15640,  15641,  15784,  15785,  15786,  15787,  18005,
         18006,  18007,  18008,  18009,  18010,  18011,  19591,  19592,  19593,
         19594,  19595,  19596,  19597,  20891,  20892,  20893,  20894,  20895,
         20896,  20897,  20910

In [43]:
train_data.trajectory_data[108616]

tensor([-1.9516, -1.8614, -1.2859, -1.5344, -2.5449, -1.3733,  0.0417, -0.0683,
         0.2720], device='cuda:2', dtype=torch.float64)

In [38]:
train_data.cumulative_index

tensor([        0.,        24.,        51.,  ...,    108597.,    108627.,
           108648.])

In [40]:
import bisect
rowId = 51
pos = bisect.bisect_left(train_data.cumulative_index.numpy(), rowId)
if train_data.cumulative_index[pos] < rowId:
    pos -= 1
print ('Trajectory containing row index : {} = {}'.format(rowId, pos))

Trajectory containing row index : 51 = 2


In [39]:
print (train_data.trajectory_index.shape)
print (train_data.trajectory_data[torch.where(train_data.trajectory_index[:, 0] == 2)[0]])

torch.Size([108678, 2])
tensor([[    -2.0725,     -0.2597,     -2.4366,     -0.4136,     -1.5639,
             -1.9923,     -0.0216,     -0.0182,      0.1102],
        [    -2.0942,     -0.2788,     -2.3412,     -0.4136,     -1.5639,
             -1.9923,      0.0004,      0.0007,      0.0149],
        [    -2.0941,     -0.2784,     -2.3145,     -0.4136,     -1.5639,
             -1.9923,      0.0768,     -0.0581,      0.0034],
        [    -2.0205,     -0.3350,     -2.3108,     -0.4136,     -1.5639,
             -1.9923,      0.0795,     -0.0600,      0.0149],
        [    -1.9418,     -0.3948,     -2.2961,     -0.4136,     -1.5639,
             -1.9923,      0.0772,     -0.0586,      0.0154],
        [    -1.8644,     -0.4535,     -2.2807,     -0.4136,     -1.5639,
             -1.9923,      0.0761,     -0.0584,      0.0152],
        [    -1.7881,     -0.5119,     -2.2655,     -0.4136,     -1.5639,
             -1.9923,      0.0762,     -0.0584,      0.0152],
        [    -1.7119,   

In [29]:
def trajectory_goal_error(trajectory):
    return torch.abs(trajectory[-1, :3]-trajectory[-1, 3:6])

In [30]:
traj_errors = torch.zeros(5000, 3)
for i in range(5000):
    traj_errors[i] = trajectory_goal_error(train_data.trajectory_data[torch.where(train_data.trajectory_index[:, 0] == i)[0]])

In [16]:
trajectory = train_data.trajectory_data[torch.where(train_data.trajectory_index == 0)[0]]

In [17]:
print (trajectory[-1, :3])
print (trajectory[-1, 3:6])
print (trajectory_goal_error(trajectory))

tensor([-2.9732, -1.6935, -1.9150], device='cuda:2', dtype=torch.float64)
tensor([-1.0334,  0.0324, -2.4570], device='cuda:2', dtype=torch.float64)
tensor([1.9398, 1.7259, 0.5420], device='cuda:2', dtype=torch.float64)


In [31]:
print(torch.min(traj_errors))
print(torch.max(traj_errors))
print (torch.where(traj_errors > 0.1)[0])
# print (traj_errors[torch.where(traj_errors > 0.15)[0]])

tensor(    0.0000)
tensor(0.0995)
tensor([], dtype=torch.int64)
